In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

In [4]:
df = pd.read_csv('data/data-ready.csv')

targets = ["z1", "z2", "z3"]

df.drop("DateTime", inplace=True, axis=1)

# split train and test
X = df.drop(columns=targets, axis=1)
y1 = df['z1']
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)

y2 = df['z2']
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)

y3 = df['z3']
X3_train, X3_test, y3_train, y3_test = train_test_split(X, y3, test_size=0.2, random_state=42)

In [5]:
from sklearn.model_selection import RepeatedKFold, cross_val_score

# fit an xgboost model to the data
model = XGBRegressor()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force scores to be positive
scores = np.abs(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()))

Mean MAE: 835.390 (12.348)


In [6]:
X1_train.describe()

,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Year,Month,Day,Hour,IsWeekend,IsNight,HourCos,HourSin,MonthCos,MonthSin,DayCos,DaySin
count,41932.000000,41932.000000,41932.000000,41932.000000,41932.000000,41932.0,41932.000000,41932.000000,41932.000000,41932.000000,41932.000000,4.193200e+04,4.193200e+04,4.193200e+04,4.193200e+04,41932.000000,4.193200e+04
mean,18.813952,68.257074,1.957745,183.606466,75.203026,2017.0,6.512496,15.674068,11.493370,0.285725,0.457574,-3.080794e-03,2.352280e-03,-6.059016e-03,-5.154925e-03,-0.021557,1.627529e-03
std,5.794997,15.522747,2.348395,264.690512,124.193888,0.0,3.438807,8.776602,6.907445,0.451764,0.498203,7.070757e-01,7.071441e-01,7.078056e-01,7.063793e-01,0.699937,7.138938e-01
min,3.247000,11.340000,0.050000,0.004000,0.011000,2017.0,1.000000,1.000000,0.000000,0.000000,0.000000,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-0.994869,-9.987165e-01
25%,14.430000,58.290000,0.078000,0.062000,0.122000,2017.0,4.000000,8.000000,6.000000,0.000000,0.000000,-7.071068e-01,-7.071068e-01,-8.660254e-01,-8.660254e-01,-0.758758,-7.247928e-01
50%,18.810000,69.850000,0.086000,5.762000,5.064500,2017.0,7.000000,16.000000,11.000000,0.000000,0.000000,-1.836970e-16,1.224647e-16,-1.836970e-16,-2.449294e-16,-0.050649,-2.449294e-16
75%,22.880000,81.400000,4.915000,323.000000,101.300000,2017.0,10.000000,23.000000,17.000000,1.000000,1.000000,7.071068e-01,7.071068e-01,5.000000e-01,5.000000e-01,0.688967,7.247928e-01
max,39.780000,94.800000,6.483000,1163.000000,936.000000,2017.0,12.000000,31.000000,23.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,9.987165e-01


### Testing Decision Trees

In [7]:
X_train = [X1_train, X2_train, X3_train]
X_test = [X1_test, X2_test, X3_test]
y_train = [y1_train, y2_train, y3_train]
y_test = [y1_test, y2_test, y3_test]

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [9]:
models = []
predictions = []
for i in range(3):
    model = DecisionTreeRegressor()
    model.fit(X_train[i], y_train[i])

    prediction = model.predict(X_test[i])
    score = r2_score(y_test[i], prediction)

    print(f"------ zone {i}: r2 score: {score}")

    models.append(model)
    predictions.append(prediction)



------ zone 0: r2 score: 0.9662874671628542
------ zone 1: r2 score: 0.9713828947307376
------ zone 2: r2 score: 0.9858790991472575


In [10]:
df["total"] = df["z1"] + df["z2"] + df["z3"]

targetT = df["total"]
XT = df.drop(columns=["z1", "z2", "z3", "total"], axis=1)


XT_train, XT_test, yT_train, yT_test = train_test_split(XT, targetT, test_size=0.2, random_state=42)

model = DecisionTreeRegressor()
model.fit(XT_train, yT_train)

prediction = model.predict(XT_test)
score = r2_score(yT_test, prediction)

print(f"------ total: r2 score: {score}")

------ total: r2 score: 0.974651392726841


### Testing Random Forests

In [11]:
from sklearn.ensemble import RandomForestRegressor

In [12]:
models_rf = []
predictions_rf = []
for i in range(3):
    model = RandomForestRegressor()
    model.fit(X_train[i], y_train[i])

    prediction = model.predict(X_test[i])
    score = r2_score(y_test[i], prediction)

    print(f"------ zone {i}: r2 score: {score}")

    models_rf.append(model)
    predictions_rf.append(prediction)

------ zone 0: r2 score: 0.98298048315401
------ zone 1: r2 score: 0.9851269727525681
------ zone 2: r2 score: 0.9925045274490086


In [14]:
model_rf = RandomForestRegressor()
model_rf.fit(XT_train, yT_train)

prediction_rf = model_rf.predict(XT_test)
score = r2_score(yT_test, prediction_rf)

print(f"------ total: r2 score: {score}")

------ total: r2 score: 0.9869350129047934


# Next:
- cross validation
- more optimization of models + more fine tuning
- group by minute, hour, day